In [121]:
import warnings
warnings.filterwarnings('ignore')
import os
import random
import json
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [122]:
from scipy import stats
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [3]:
with open('vegas_contents.pkl','rb') as f:
    vegas_contents = pickle.load(f)

In [34]:
vegas_contents.head()

,business_id,name,address,postal_code,latitude,longitude,categories,sep_categories,Dominant Ids,Percent,Topics,Features,stars,Combined Features
0,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",89103,36.099361,-115.226636,"Pizza, Salad, Burgers, Restaurants","[Pizza, Salad, Burgers, Restaurants]","[6, 1, 0]","[0.71840245, 0.10773659, 0.06156651]","[Good Ambiance / Service, Fastfood / American,...","[Pizza, Salad, Burgers, Ambiance / Good Servic...",3.694118,Pizza Salad Burgers Ambiance / Good Service Fa...
1,AN0bWhisCf6LN9eHZ7DQ3w,Los Olivos Ristorante,3759 E Desert Inn Rd,89121,36.129178,-115.092483,"Restaurants, Italian","[Restaurants, Italian]","[2, 6, 0]","[0.44978717, 0.2163004, 0.09988998]","[Dinner, Good Ambiance / Service, Continental]","[Italian, Steakhouse / Dinner, Ambiance / Good...",4.807143,Italian Steakhouse / Dinner Ambiance / Good Se...
2,AtD6B83S4Mbmq0t7iDnUVA,Veggie House,"5115 Spring Mountain Rd, Ste 203",89146,36.125569,-115.210911,"Restaurants, Specialty Food, Japanese, Sushi B...","[Restaurants, Specialty Food, Japanese, Sushi ...","[4, 0, 3]","[0.30285373, 0.2803057, 0.13361645]","[Fusion / Buffet, Continental, Snack / Tacos]","[Specialty Food, Japanese, Sushi Bars, Dim Sum...",4.523707,Specialty Food Japanese Sushi Bars Dim Sum Veg...
3,oUX2bYbqjqST-urKbOHG6w,Loftti Cafe,"7729 S Rainbow Blvd, Ste 9B",89139,36.047942,-115.244167,"Sandwiches, Shaved Ice, Coffee & Tea, Desserts...","[Sandwiches, Shaved Ice, Coffee & Tea, Dessert...","[0, 6, 5]","[0.39396736, 0.19578542, 0.14329998]","[Continental, Good Ambiance / Service, Exotic]","[Sandwiches, Shaved Ice, Coffee & Tea, Dessert...",4.277512,Sandwiches Shaved Ice Coffee & Tea Desserts Ca...
4,FiW6w5nmhlUoJAyNofb4jg,Fruits and Roots,5020 Blue Diamond Rd,89139,36.032122,-115.210267,"Coffee & Tea, Food Stands, Food, Restaurants, ...","[Coffee & Tea, Food Stands, Food, Restaurants,...","[0, 3, 5]","[0.49596417, 0.12588032, 0.09800566]","[Continental, Snack / Tacos, Exotic]","[Coffee & Tea, Food Stands, Juice Bars & Smoot...",4.296296,Coffee & Tea Food Stands Juice Bars & Smoothie...


In [5]:
# Reading the json file
with open('rec.json', 'r') as fp:
    user_ids = json.load(fp)
with open('zip_codes.json', 'r') as fp:
    zips = json.load(fp)

In [6]:
zip_codes = vegas_contents['postal_code'].unique().tolist()

In [90]:
zip_site = [item['fields']['zip'] for item in zips]

In [91]:
[item for item in zip_codes if item not in zip_site]

['89183', '89178', '89169', '89166', '89179']

In [ ]:
manual_locations = [('89183',[35.9987, -115.1593]),
                   ('89169',[36.1295, -115.1326]),
                   ('89178',[36.0457, -115.3134]),
                   ('89166',[36.3192, -115.3683]),
                   ('89179',[35.9919, -115.2555])]

In [227]:
[(item,item) for item in list(map(str,sorted([int(item) for item in sorted(locations.keys())])))]

[('89014', '89014'),
 ('89019', '89019'),
 ('89030', '89030'),
 ('89031', '89031'),
 ('89032', '89032'),
 ('89052', '89052'),
 ('89084', '89084'),
 ('89101', '89101'),
 ('89102', '89102'),
 ('89103', '89103'),
 ('89104', '89104'),
 ('89106', '89106'),
 ('89107', '89107'),
 ('89108', '89108'),
 ('89109', '89109'),
 ('89110', '89110'),
 ('89111', '89111'),
 ('89113', '89113'),
 ('89114', '89114'),
 ('89115', '89115'),
 ('89117', '89117'),
 ('89118', '89118'),
 ('89119', '89119'),
 ('89120', '89120'),
 ('89121', '89121'),
 ('89122', '89122'),
 ('89123', '89123'),
 ('89128', '89128'),
 ('89129', '89129'),
 ('89130', '89130'),
 ('89131', '89131'),
 ('89134', '89134'),
 ('89135', '89135'),
 ('89136', '89136'),
 ('89139', '89139'),
 ('89141', '89141'),
 ('89142', '89142'),
 ('89143', '89143'),
 ('89144', '89144'),
 ('89145', '89145'),
 ('89146', '89146'),
 ('89147', '89147'),
 ('89148', '89148'),
 ('89149', '89149'),
 ('89152', '89152'),
 ('89156', '89156'),
 ('89158', '89158')]

In [120]:
locations.keys()

dict_keys(['89148', '89147', '89103', '89113', '89145', '89117', '89158', '89123', '89032', '89115', '89119', '89130', '89102', '89120', '89108', '89118', '89110', '89107', '89101', '89131', '89111', '89122', '89014', '89141', '89104', '89031', '89139', '89146', '89134', '89030', '89128', '89152', '89084', '89052', '89114', '89121', '89136', '89106', '89129', '89019', '89156', '89144', '89109', '89149', '89142', '89143', '89135'])

In [7]:
locations = {item['fields']['zip']:item['fields']['geopoint'] for item in zips if item['fields']['zip'] in zip_codes}

In [21]:
topics_map = {0: 'Continental',
              1: 'Fastfood / American',
              2: 'Dinner',
              3: 'Snack / Tacos',
              4: 'Fusion / Buffet',
              5: 'Exotic',
              6: 'Good Ambiance / Service',
              7: 'Hotel/ Bar/ Night life'
             }  

def map_dominant(num):
    return topics_map[num]

In [23]:
vegas_contents['Topics'] = vegas_contents['Dominant Ids'].apply(lambda x: list(map(map_dominant,x)))

In [55]:
# approximate radius of earth in km
def distance(coord1,coord2):
    R = 6373.0
    lat1,lon1 = coord1
    lat2,lon2 = coord2
    lat1,lon1 = np.radians(lat1), np.radians(lon1)
    lat2,lon2 = np.radians(lat2), np.radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    dist = R*c/1.6
    return dist

coord1 = (36.2296756, -115.0122287)
coord2 = (36.2006374, -115.2251681)
print(f"The distance between the two is about {int(distance(coord1,coord2)):2} miles.")

The distance between the two is about 12 miles.


In [9]:
zip_std = '89134'
for zip in ['89148','89147','89113','89139']:
    if (distance(locations[zip_std],locations[zip])) < 20.0:
        print(f'The distance from {zip_std} to {locations[zip]} is {(distance(locations[zip_std],locations[zip])):9.3f} miles')

The distance from 89134 to [36.049095, -115.28485] is    10.914 miles
The distance from 89134 to [36.113059, -115.27343] is     6.720 miles
The distance from 89134 to [36.067495, -115.26039] is     9.960 miles
The distance from 89134 to [36.031873, -115.20342] is    13.466 miles


In [25]:
vegas_contents['Features'] = vegas_contents['Features'].apply(lambda x: [item for item in x if item not in ['Restaurants','Food']])

In [26]:
vegas_contents['Combined Features'] = (vegas_contents['Features']).apply(lambda x: ' '.join(x))

In [27]:
categories_list = vegas_contents['sep_categories'].values.flatten()
categories_list = [item for sublist in categories_list for item in sublist]
unique_categories = sorted(list(set(categories_list)))

In [28]:
topic_list = vegas_contents['Topics'].values.flatten()
topic_list = [item for sublist in topic_list for item in sublist]
unique_topics = sorted(list(set(topic_list)))

In [35]:
def get_restaurants_by_preference(df,categories_list):
    
    store_restaurants = []
    
    df_rest = pd.DataFrame(columns=vegas_contents.columns)
    for category in categories_list:        
        category_restaurants = df[df['sep_categories'].apply(lambda x: True if category in x else False)]
        rest_list = category_restaurants[['business_id','stars']].groupby('business_id').mean().sort_values(by=['stars'],ascending=False).index.tolist()
        df_categ = vegas_contents[vegas_contents['business_id'].apply(lambda x: True if x in rest_list else False)]
        df_rest = pd.concat([df_rest,df_categ])  
    for index,row in df_rest.iterrows():
        restaurant_details = {}
        restaurant_details['Name'] = row['name']
        restaurant_details['Address'] = row['address']
        restaurant_details['Zip Code'] = row['postal_code']
        restaurant_details['Latitude'] = row['latitude']
        restaurant_details['Longitude'] = row['longitude']
        
        restaurant_details['Category'] = row['sep_categories']
        restaurant_details['Topics'] = row['Topics']
        restaurant_details['Stars'] = row['stars']
#         restaurant_details['additional_details'] = row['attributes']
        store_restaurants.append(restaurant_details)
    return sorted(store_restaurants,key = lambda x: x['Stars'],reverse=True)

def get_restaurants_by_topic(df,categories_list):
    
    store_restaurants = []
    
    df_rest = pd.DataFrame(columns=vegas_contents.columns)
    for category in categories_list:        
        category_restaurants = df[df['Topics'].apply(lambda x: True if category in x else False)]
        rest_list = category_restaurants[['business_id','stars']].groupby('business_id').mean().sort_values(by=['stars'],ascending=False).index.tolist()
        df_categ = vegas_contents[vegas_contents['business_id'].apply(lambda x: True if x in rest_list else False)]
        df_rest = pd.concat([df_rest,df_categ])  
    for index,row in df_rest.iterrows():
        restaurant_details = {}
        restaurant_details['Name'] = row['name']
        restaurant_details['Address'] = row['address']
        restaurant_details['Zip Code'] = row['postal_code']
        restaurant_details['Latitude'] = row['latitude']
        restaurant_details['Longitude'] = row['longitude']
        
        restaurant_details['Category'] = row['sep_categories']
        restaurant_details['Topics'] = row['Topics']
        restaurant_details['Stars'] = row['stars']
#         restaurant_details['additional_details'] = row['attributes']
        store_restaurants.append(restaurant_details)
    return sorted(store_restaurants,key = lambda x: x['Stars'],reverse=True)

In [36]:
def get_restaurants_by_usr_id(df,usr_id):
    
    store_restaurants = []
    
    df_rest = vegas_contents[vegas_contents['business_id'].isin([item[0] for item in user_ids[usr_id]])]
     
    for index,row in df_rest.iterrows():
        restaurant_details = {}
        restaurant_details['Name'] = row['name']
        restaurant_details['Address'] = row['address']
        restaurant_details['Zip Code'] = row['postal_code']
        restaurant_details['Latitude'] = row['latitude']
        restaurant_details['Longitude'] = row['longitude']
        
        restaurant_details['Category'] = row['sep_categories']
        restaurant_details['Topics'] = row['Topics']
        restaurant_details['Stars'] = row['stars']
#         restaurant_details['additional_details'] = row['attributes']
        store_restaurants.append(restaurant_details)
    return sorted(store_restaurants,key = lambda x: x['Stars'],reverse=True)

In [30]:
vegas_contents.head()

,business_id,name,address,postal_code,latitude,longitude,categories,sep_categories,Dominant Ids,Percent,Topics,Features,stars,Combined Features
0,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",89103,36.099361,-115.226636,"Pizza, Salad, Burgers, Restaurants","[Pizza, Salad, Burgers, Restaurants]","[6, 1, 0]","[0.71840245, 0.10773659, 0.06156651]","[Good Ambiance / Service, Fastfood / American,...","[Pizza, Salad, Burgers, Ambiance / Good Servic...",3.694118,Pizza Salad Burgers Ambiance / Good Service Fa...
1,AN0bWhisCf6LN9eHZ7DQ3w,Los Olivos Ristorante,3759 E Desert Inn Rd,89121,36.129178,-115.092483,"Restaurants, Italian","[Restaurants, Italian]","[2, 6, 0]","[0.44978717, 0.2163004, 0.09988998]","[Dinner, Good Ambiance / Service, Continental]","[Italian, Steakhouse / Dinner, Ambiance / Good...",4.807143,Italian Steakhouse / Dinner Ambiance / Good Se...
2,AtD6B83S4Mbmq0t7iDnUVA,Veggie House,"5115 Spring Mountain Rd, Ste 203",89146,36.125569,-115.210911,"Restaurants, Specialty Food, Japanese, Sushi B...","[Restaurants, Specialty Food, Japanese, Sushi ...","[4, 0, 3]","[0.30285373, 0.2803057, 0.13361645]","[Fusion / Buffet, Continental, Snack / Tacos]","[Specialty Food, Japanese, Sushi Bars, Dim Sum...",4.523707,Specialty Food Japanese Sushi Bars Dim Sum Veg...
3,oUX2bYbqjqST-urKbOHG6w,Loftti Cafe,"7729 S Rainbow Blvd, Ste 9B",89139,36.047942,-115.244167,"Sandwiches, Shaved Ice, Coffee & Tea, Desserts...","[Sandwiches, Shaved Ice, Coffee & Tea, Dessert...","[0, 6, 5]","[0.39396736, 0.19578542, 0.14329998]","[Continental, Good Ambiance / Service, Exotic]","[Sandwiches, Shaved Ice, Coffee & Tea, Dessert...",4.277512,Sandwiches Shaved Ice Coffee & Tea Desserts Ca...
4,FiW6w5nmhlUoJAyNofb4jg,Fruits and Roots,5020 Blue Diamond Rd,89139,36.032122,-115.210267,"Coffee & Tea, Food Stands, Food, Restaurants, ...","[Coffee & Tea, Food Stands, Food, Restaurants,...","[0, 3, 5]","[0.49596417, 0.12588032, 0.09800566]","[Continental, Snack / Tacos, Exotic]","[Coffee & Tea, Food Stands, Juice Bars & Smoot...",4.296296,Coffee & Tea Food Stands Juice Bars & Smoothie...


In [66]:
with open('restaurants.pkl', 'wb') as fp:
    pickle.dump(vegas_contents, fp)

In [38]:
topic = 'Continental'
get_restaurants_by_topic(vegas_contents,categories_list=[topic])[:2]


[{'Name': 'Brew Tea Bar',
  'Address': '7380 S Rainbow Blvd, Ste 101',
  'Zip Code': '89139',
  'Latitude': 36.0542269,
  'Longitude': -115.2423924,
  'Category': ['Restaurants',
   'Food',
   'Cafes',
   'Tea Rooms',
   'Bubble Tea',
   'Desserts'],
  'Topics': ['Continental', 'Good Ambiance / Service', 'Exotic'],
  'Stars': 4.830034129692833},
 {'Name': 'POTs',
  'Address': '1745 S Rainbow Blvd, Ste a',
  'Zip Code': '89146',
  'Latitude': 36.1520101,
  'Longitude': -115.243704,
  'Category': ['Vegan',
   'Middle Eastern',
   'Egyptian',
   'Restaurants',
   'Vegetarian'],
  'Topics': ['Continental', 'Fastfood / American', 'Good Ambiance / Service'],
  'Stars': 4.82}]

In [40]:
preference = 'Chinese'
get_restaurants_by_preference(vegas_contents,categories_list=[preference])[:2]

[{'Name': 'Chef Kennys Asian Vegan Restaurant',
  'Address': '6820 Spring Mountain Rd, Ste 111',
  'Zip Code': '89146',
  'Latitude': 36.1269461,
  'Longitude': -115.2410542,
  'Category': ['Restaurants', 'Chinese', 'Vegan', 'Sushi Bars', 'Vegetarian'],
  'Topics': ['Exotic', 'Fusion / Buffet', 'Continental'],
  'Stars': 4.646766169154229},
 {'Name': 'Sushi King',
  'Address': '3430 E Tropicana Ave, Ste 30',
  'Zip Code': '89121',
  'Latitude': 36.1006736,
  'Longitude': -115.1002334,
  'Category': ['Restaurants',
   'Asian Fusion',
   'Sushi Bars',
   'Ramen',
   'Hawaiian',
   'Japanese',
   'Chinese'],
  'Topics': ['Exotic', 'Fusion / Buffet', 'Continental'],
  'Stars': 4.627659574468085}]

In [42]:
preference1 = ['Chinese']
preference2 = ['Indian']
topic = ['Continental']
store_rest = []

In [43]:
rest_pref1 = get_restaurants_by_preference(vegas_contents,preference1)
rest_pref2 = get_restaurants_by_preference(vegas_contents,preference2)
pref_store = [rest_pref1,rest_pref2]

In [44]:
if preference1 != preference2:
    for i in range(2):
        store_rest.extend(pref_store[i][:10])
else:
    store_rest.extend(pref_store[0][:20])
store_rest.extend(get_restaurants_by_topic(vegas_contents,topic)[:10])

In [65]:
zip_code = '89129'
proximity = 20.0
for item in store_rest:
    dist = distance(locations[zip_code],locations[item['Zip Code']])    
    if dist < proximity:        
        print(f'The distance from {zip_code} is {dist:9.3f} miles')

The distance from 89129 is     7.098 miles
The distance from 89129 is    13.285 miles
The distance from 89129 is     8.230 miles
The distance from 89129 is     7.098 miles
The distance from 89129 is    14.584 miles
The distance from 89129 is     8.233 miles
The distance from 89129 is     8.999 miles
The distance from 89129 is    12.655 miles
The distance from 89129 is    13.285 miles
The distance from 89129 is     8.230 miles
The distance from 89129 is    12.655 miles
The distance from 89129 is     8.230 miles
The distance from 89129 is     9.761 miles
The distance from 89129 is     8.233 miles
The distance from 89129 is    12.826 miles
The distance from 89129 is    12.826 miles
The distance from 89129 is     0.000 miles
The distance from 89129 is    12.826 miles
The distance from 89129 is    16.148 miles
The distance from 89129 is     8.230 miles
The distance from 89129 is    14.584 miles
The distance from 89129 is     7.098 miles
The distance from 89129 is    14.584 miles
The distanc

In [71]:
[item for item in store_rest if distance(locations[zip_code],locations[item['Zip Code']]) < proximity][:2]

[{'Name': 'Chef Kennys Asian Vegan Restaurant',
  'Address': '6820 Spring Mountain Rd, Ste 111',
  'Zip Code': '89146',
  'Latitude': 36.1269461,
  'Longitude': -115.2410542,
  'Category': ['Restaurants', 'Chinese', 'Vegan', 'Sushi Bars', 'Vegetarian'],
  'Topics': ['Exotic', 'Fusion / Buffet', 'Continental'],
  'Stars': 4.646766169154229},
 {'Name': 'Sushi King',
  'Address': '3430 E Tropicana Ave, Ste 30',
  'Zip Code': '89121',
  'Latitude': 36.1006736,
  'Longitude': -115.1002334,
  'Category': ['Restaurants',
   'Asian Fusion',
   'Sushi Bars',
   'Ramen',
   'Hawaiian',
   'Japanese',
   'Chinese'],
  'Topics': ['Exotic', 'Fusion / Buffet', 'Continental'],
  'Stars': 4.627659574468085}]

In [94]:
import folium
from folium import plugins
from folium.plugins import HeatMap

In [149]:
vegas_coordinates = (36.1699, -115.1398)
folium_map = folium.Map(location=vegas_coordinates,width=1000,height=800,zoom_start=11)

In [150]:
for restaurant in store_rest:
    coordinates = restaurant['Latitude'],restaurant['Longitude']
    folium.Marker(coordinates,popup=(restaurant['Name'],restaurant['Category'])).add_to(folium_map)

In [151]:
data = [(restaurant['Latitude'],restaurant['Longitude']) for restaurant in store_rest]
HeatMap(data).add_to(folium.FeatureGroup(name='Heat Map').add_to(folium_map))


In [152]:
folium_map

In [67]:
with open('restaurants.pkl','rb') as f:
    restaurants = pickle.load(f)

In [131]:
vect = CountVectorizer()
tfidf = TfidfVectorizer()

In [126]:
vect_matrix = vect.fit_transform(vegas_contents['Combined Features'])

In [132]:
tfidf_matrix = tfidf.fit_transform(vegas_contents['Combined Features'])

In [127]:
vect_matrix.shape

(2132, 395)

In [133]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [135]:
cosine_sim[0]

array([1.        , 0.2896276 , 0.03925627, ..., 0.17515934, 0.17049532,
       0.20940573])

In [136]:
vegas_contents.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131],
           dtype='int64', length=2132)

In [157]:
vegas_contents['business_id']

0       Yr_w9lakJrKMyEG_hI6zbA
1       AN0bWhisCf6LN9eHZ7DQ3w
2       AtD6B83S4Mbmq0t7iDnUVA
3       oUX2bYbqjqST-urKbOHG6w
4       FiW6w5nmhlUoJAyNofb4jg
                 ...          
2127    CxZ_qwVlgd-DIrb_a3N8jw
2128    K19uBHydrYAsfX3CPnFp-w
2129    P3xSYeiJFkbdBwpnEncPTg
2130    ak-f8IV0ZdDeI4oYnjA1Pw
2131    PIXl4WvAqjl2w9Lj81zXvQ
Name: business_id, Length: 2132, dtype: object

In [158]:
businesses = vegas_contents['business_id']
indices = pd.Series(businesses.index,index = vegas_contents['business_id'])

In [195]:
def content_recommendations(business):
    idx = indices[business]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    biz_indices = [i[0] for i in sim_scores]
    return businesses.iloc[biz_indices].values.tolist()

In [196]:
content_recommendations('d4qwVw4PcN-_2mK2o1Ro1g')

['d4qwVw4PcN-_2mK2o1Ro1g',
 '2XkVexeY_8zyc9nriSElQg',
 '2IvrdAb6zdxr3ZqplqJHbg',
 'w55sCX7ZkLIJNhl_RhwjEg',
 'GOJJynE4H3i5XghQiy24Zw',
 'WcGKG8_rDVBVx6lSTlWiDA',
 'pLMPpaJ7whp86uL_3ezwMg',
 'BQJokxJ4y4s57i7Mn65pvA',
 'Oto60yDwk1z72WmfWEYrjg',
 'TzpqNdtAZHXuIKf2xS0o2A',
 'CiYLq33nAyghFkUR15pP-Q',
 'c-NXKTJ0jrrusTPxJAUwvA',
 'Xj7DIGRHEchJ-VVdISazQQ',
 '8KEB1lcjKlIVSd6uA8JB9w',
 'YaTlfyPieOXwHH16wzieFg',
 'MZRqxvBkJLjKcoOjNz7yLA',
 'Nr5TGaBoGraxbZBCzKyvuA',
 'Zrx25j1M794Nh8fUGB8E9A',
 'UGiMMw54qYxac0eE_cBbaA',
 'VFcx35QDd_kZgLT4UDV8xg']

In [162]:
with open('vegas_reviews.pkl','rb') as f:
    vegas_reviews = pickle.load(f)

In [165]:
type(vegas_reviews.iloc[0]['date'])

str

In [166]:
import datetime

In [167]:
vegas_reviews['date'] = vegas_reviews['date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [175]:
vegas_reviews_open = vegas_reviews[vegas_reviews['business_id'].isin(vegas_contents['business_id'])]

In [176]:
vegas_reviews_open.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
16,bVTjZgRNq8ToxzvtiVrqMA,2hRe26HSCAWbFRn5WChK-Q,d4qwVw4PcN-_2mK2o1Ro1g,1.0,0,0,0,10pm on a super bowl Sunday and they're alread...,2015-02-02 06:28:00
21,TeX0eOIeZb_wpo86rCedXQ,RR-2nouBn408e3djxC470g,d4qwVw4PcN-_2mK2o1Ro1g,5.0,0,0,0,Holy heck this place is amazing. I love their ...,2017-02-09 04:25:03
24,XvLG7ReC8JZmBltOLJzfcA,-Co-ReNx_lXT1xL_Rr0B2g,XZbuPXdyA0ZtTu3AzqtQhg,4.0,0,1,0,"As the previous person posted, what more can r...",2009-10-13 09:50:48
30,WGbtqQMeJwTVg6DFb156EQ,K5MSNpwRf0xKQSaC6gK9Cw,PL3cimEUfNHlenOGSOAdJg,4.0,3,0,1,"This was our choice, by default, for New Year'...",2017-01-02 05:24:56
40,TkcxD1A4N_9kRZwGhUjDKA,bgG0jJyUXta_kczGrQW2Rw,7EZ4Eu7YJ1ltRCC5jXFJrQ,1.0,1,0,0,There wasn't one specific thing that made this...,2013-04-10 16:19:12


In [179]:
last_restaurants = vegas_reviews_open[['user_id','business_id','date']].merge(vegas_reviews_open.groupby('user_id')['date'].max(),on=['user_id','date'])

In [182]:
user_ids.keys()

dict_keys(['aWXGwath_dSGPwNdFiy9BQ', 'WM7MFrRP-7YFuGBAJ6quRQ', 'Q7IoE5m2heQKThuVq3SYFA', '5wT0vDycEzgq95yVCZ60Aw', 'RQlnSCjuqMnhR3Qk6j4KoA', '_7PfR6Tvh2xTbiVi1GELoQ', 'bLbSNkLggFnqwNNzzq-Ijw', 'keBv05MsMFBd0Hu98vXThQ', 'B1829_hxXSEpDPEDJtYeIw', 'rt3PC7WCgCKsoufmQJELfw'])

In [189]:
usr_biz = [(usr,last_restaurants[last_restaurants['user_id'].isin( [usr])]['business_id'].iloc[0]) for usr in user_ids.keys()]
usr_biz

[('aWXGwath_dSGPwNdFiy9BQ', '0w3BdEOxvwKTpypwbN5kiQ'),
 ('WM7MFrRP-7YFuGBAJ6quRQ', 'Ia1JlEo7UUyZtmqRe1K1Pw'),
 ('Q7IoE5m2heQKThuVq3SYFA', '1aEk5vowhhTa-C3EqxdEFw'),
 ('5wT0vDycEzgq95yVCZ60Aw', 'JDZ6_yycNQFTpUZzLIKHUg'),
 ('RQlnSCjuqMnhR3Qk6j4KoA', '8-8btzuYZCqEVOPwk0YYfA'),
 ('_7PfR6Tvh2xTbiVi1GELoQ', 'D_24gRsnae7guxJE8gLgtw'),
 ('bLbSNkLggFnqwNNzzq-Ijw', 'N0CoaG3cBrTGm2ecLYSzqA'),
 ('keBv05MsMFBd0Hu98vXThQ', 'mDoLWEOFmxcQ5TAPqxoJLg'),
 ('B1829_hxXSEpDPEDJtYeIw', 'Mh84_j7YmRrAcm0DWV4bUw'),
 ('rt3PC7WCgCKsoufmQJELfw', 'e3PBfbtXUjMziX6XGsOMoA')]

In [197]:
store_restaurants_content = {item[0]:content_recommendations(item[1]) for item in usr_biz}

In [199]:
with open('rec_content.json', 'w') as fp:
    json.dump(store_restaurants_content, fp)

In [203]:
test_user = 'aWXGwath_dSGPwNdFiy9BQ'

In [206]:
store_restaurants_content[test_user]

['Po0QGzgwFrfnGJEi_akCjA',
 'wFaVu_D7LiwAeDl_TxCf8g',
 'yfxDa8RFOvJPQh0rNtakHA',
 'buTa1akbH0pO_P10v99_eA',
 'TW58a4UkMcksdfQ7znUqsA',
 'OXBROmBWuDhRhd0BZ7zXJA',
 '7iuruLs-q-_RW06IbsBxZw',
 '8-8btzuYZCqEVOPwk0YYfA',
 'MQI1rSTE2LGrrsPPgenCrA',
 'ygaOvp0PLBYaYeN9cZAlGg',
 'eaNenRk_liZBERFFLCXqqQ',
 'G6w_9uzW4o3Oyb3z8oOZyA',
 'FQB3VXrWYM_m_XrzwRozAg',
 'EkuSy_kM8dpGrlb2pTxCBw',
 '8F-UdXe_YylX3k0L1SEiRg',
 'm51uL1LZhKYf2y_yG4NEng',
 'N4t78qF2C5jqYBjNr0j3_w',
 'FSnqO-YWUdRb0Xc95nXbvw',
 'iRwT2pBa87Y54q8nqfNAzw',
 'BulTr0Cdk-0RPADpM1oFMw']

In [205]:
[item[0] for item in user_ids[test_user]]

['B-9IuVXitKKwXj0fXUxczQ',
 'G4hjhtA_wQ-tSOGpgGlDjw',
 'cOqd9ou3uf-1k3OWFo-_CQ',
 'cePE3rCuUOVSCCAHSjWxoQ',
 'Jjwa-WCK73efAwVZRksWSw',
 'vOMDU31gdylrzBhAKC9QbA',
 '9P23-V64kYz3trn9ecaJJA',
 '8fFTJBh0RB2EKG53ibiBKw',
 'iX2c4IzH_L5F6FM5GD3SUQ',
 '3pSUr_cdrphurO6m1HMP9A']

In [211]:
def get_restaurants_by_usr_id(df,usr_id):
    
    store_restaurants = []
    
    df_colab = vegas_contents[vegas_contents['business_id'].isin([item[0] for item in user_ids[usr_id]])]
    df_content = vegas_contents[vegas_contents['business_id'].isin([item for item in store_restaurants_content[usr_id]])]
    df_rest = pd.concat([df_colab,df_content])
     
    for index,row in df_rest.iterrows():
        restaurant_details = {}
        restaurant_details['Name'] = row['name']
        restaurant_details['Address'] = row['address']
        restaurant_details['Zip Code'] = row['postal_code']
        restaurant_details['Latitude'] = row['latitude']
        restaurant_details['Longitude'] = row['longitude']
        
        restaurant_details['Category'] = row['sep_categories']
        restaurant_details['Topics'] = row['Topics']
        restaurant_details['Stars'] = row['stars']
#         restaurant_details['additional_details'] = row['attributes']
        store_restaurants.append(restaurant_details)
    return sorted(store_restaurants,key = lambda x: x['Stars'],reverse=True)

In [218]:
store = get_restaurants_by_usr_id(vegas_contents,test_user)
[item for item in store if distance(locations[zip_code],locations[item['Zip Code']]) < proximity]

[{'Name': 'POTs',
  'Address': '1745 S Rainbow Blvd, Ste a',
  'Zip Code': '89146',
  'Latitude': 36.1520101,
  'Longitude': -115.243704,
  'Category': ['Vegan',
   'Middle Eastern',
   'Egyptian',
   'Restaurants',
   'Vegetarian'],
  'Topics': ['Continental', 'Fastfood / American', 'Good Ambiance / Service'],
  'Stars': 4.82},
 {'Name': 'Carnitas King',
  'Address': '6030 W Windmill Ln',
  'Zip Code': '89139',
  'Latitude': 36.0426337,
  'Longitude': -115.2245316,
  'Category': ['Mexican', 'Restaurants'],
  'Topics': ['Snack / Tacos', 'Good Ambiance / Service', 'Continental'],
  'Stars': 4.813559322033898},
 {'Name': 'Bajamar Seafood & Tacos',
  'Address': '1615 S Las Vegas Blvd',
  'Zip Code': '89104',
  'Latitude': 36.1517421042,
  'Longitude': -115.1518118808,
  'Category': ['Fast Food',
   'Dive Bars',
   'Bars',
   'Tacos',
   'Seafood',
   'Nightlife',
   'Mexican',
   'Restaurants'],
  'Topics': ['Snack / Tacos', 'Dinner', 'Fusion / Buffet'],
  'Stars': 4.7995049504950495},
 {